In [4]:
import os
import re
import pandas as pd
import numpy as np

In [5]:
dir_path = r"E:/mini project bristol/JPMorgan_Set01/JPMorgan_Set01/Tapes"
file_ls = os.listdir(dir_path)  #get every name of the file
file_ls[:5]

['UoB_Set01_2025-01-02tapes.csv',
 'UoB_Set01_2025-01-03tapes.csv',
 'UoB_Set01_2025-01-06tapes.csv',
 'UoB_Set01_2025-01-07tapes.csv',
 'UoB_Set01_2025-01-08tapes.csv']

In [6]:
len(file_ls)

125

In [7]:
#found the max and min deal price
def detailprice(name):
    tapes=pd.read_csv(r"E:/mini project bristol/JPMorgan_Set01/JPMorgan_Set01/Tapes/"+name+"",header=None,names=["Times","price","volume"])
    tapes['price'] = pd.to_numeric(tapes['price'], errors='coerce')
    max_price=max(tapes.price)
    min_price=min(tapes.price)
    open_price=tapes["price"].iloc[0]
    close_price=tapes["price"].iloc[-1]
    return max_price,min_price,open_price,close_price

In [8]:
max_price=[]
min_price=[]
open_price=[]
close_price=[]
for i in file_ls:
    max_price.append(detailprice(i)[0])
    min_price.append(detailprice(i)[1])
    open_price.append(detailprice(i)[2])
    close_price.append(detailprice(i)[3])
type(max_price)

list

In [9]:
data_pattern=r'\d{4}-\d{2}-\d{2}'# found the date in the file
date=[]
for i in file_ls:
    match=re.search(data_pattern,i)
    date.append(match.group())
date=pd.to_datetime(date)


In [14]:
file_ls_series = pd.Series(file_ls)
results_df=[]
def apply_detailprice(filename):
    return detailprice(filename)  # Make sure that this function returns a tuple or list of four values

results_df = file_ls_series.apply(apply_detailprice).apply(pd.Series)
results_df.columns = ['max_price', 'min_price', 'open_price', 'close_price']
date=pd.Series(date)
results_df.insert(0,"date",date)
results_df.head(5)



,date,max_price,min_price,open_price,close_price
0,2025-01-02,303,243,267,288
1,2025-01-03,317,270,281,291
2,2025-01-06,346,276,278,327
3,2025-01-07,388,317,336,369
4,2025-01-08,374,303,365,311


In [15]:
start_date,end_date=date.min(),date.max()# create a new dataframe which have all date in this range
date_range=pd.date_range(start=start_date,end=end_date)
date_range=pd.Series(date_range)
df_full = pd.DataFrame(date_range, columns=['date'])
df_full.head(5)

,date
0,2025-01-02
1,2025-01-03
2,2025-01-04
3,2025-01-05
4,2025-01-06


In [16]:
df_merged=pd.merge(df_full,results_df,on="date",how="left")# left join 2 df
df_merged.head(5)

,date,max_price,min_price,open_price,close_price
0,2025-01-02,303.0,243.0,267.0,288.0
1,2025-01-03,317.0,270.0,281.0,291.0
2,2025-01-04,NaN,NaN,NaN,NaN
3,2025-01-05,NaN,NaN,NaN,NaN
4,2025-01-06,346.0,276.0,278.0,327.0


In [19]:
df_merged.to_csv("E:/mini project bristol/processed data(LOBS,tapes)/processed_tapes.csv")